In [1]:
import numpy as np
import numpy.matlib

import matplotlib.pyplot as plt
import matplotlib.pylab as plt
from matplotlib.pyplot import figure

import scipy.integrate
from scipy.integrate import solve_ivp
from scipy.sparse import csr_matrix, triu, spdiags
from scipy.sparse.linalg import spsolve, splu
import scipy.sparse
import time

### Problem 2

### A4

In [2]:
# delta x or delta y 
n = 128
L = 10
l = np.linspace(-L, L, n+1)
dl = l[1]-l[0]

In [3]:
def A(m):

    # m = 64 # N value in x and y directions
    n = m*m # total size of matrix

    e1 = np.ones(n) # vector of ones
    Low1 = np.tile(np.concatenate((np.ones(m-1), [0])), (m,)) # Lower diagonal 1
    Low2 = np.tile(np.concatenate(([1], np.zeros(m-1))), (m,)) #Lower diagonal 2
                                        # Low2 is NOT on the second lower diagonal,
                                        # it is just the next lower diagonal we see
                                        # in the matrix.

    Up1 = np.roll(Low1, 1) # Shift the array for spdiags
    Up2 = np.roll(Low2, m-1) # Shift the other array

    A = scipy.sparse.spdiags([e1, e1, Low2, Low1, -4*e1, Up1, Up2, e1, e1],
                             [-(n-m), -m, -m+1, -1, 0, 1, m-1, m, (n-m)], n, n,format = 'csr')
    
    return A

In [4]:
A = A(128)
A[0,0] = 2
A = A/(dl**2)

In [5]:
A4 = np.copy(A.toarray())

### A5

In [6]:
def B(m):
    # m = 5
    n = m*m
    e1_B = np.ones((n)).T # vector of ones

    d_B = np.array([-((m**2)-m), - m, m, (m**2)-m])
    Bin_B = np.array([e1_B, -e1_B, e1_B, -e1_B])


    B = spdiags(Bin_B,d_B, n, n, format = 'csc')
    B = B/(2 * dl)
    return B

In [7]:
B = B(128)

In [8]:
A5 = np.copy(B.toarray())

### A6

In [9]:
def C(m):
    # m = 5 
    n = m*m

    e1_C = np.ones(n) # vector of ones
    Low1_C = np.tile(np.concatenate((np.ones(m-1), [0])), (m,)) # Lower diagonal 1
    Low2_C = np.tile(np.concatenate(([1], np.zeros(m-1))), (m,)) #Lower diagonal 2

    Up1_C = np.roll(Low1_C, 1) 
    Up2_C = np.roll(Low2_C, m-1)

    C = scipy.sparse.spdiags([Low2_C, -Low1_C, Up1_C, -Up2_C],
                            [-m+1, -1, 1, m-1], n, n,format = 'csc')
    C = C/(2 * dl)
    return C

In [10]:
C = C(128)

In [11]:
A6 = np.copy(C.toarray())

### A7

In [12]:
L = 10
xp = [0, 4]

# time
t_evals = np.linspace(0,4,9)
dt = 0.5

In [15]:
x = np.linspace(-10, 10, 128, endpoint=False)
y = np.linspace(-10, 10, 128, endpoint=False)
# k = np.exp(-2*(x**2) - ((y**2)/20))

In [16]:
w0_func = lambda x,y: np.exp((-2*(x**2))-((y**2)/20))
w0=np.zeros(len(x)**2)
k=0
for i in x:
    for j in y:
        w0[k]= w0_func(i,j)
        k=k+1

In [17]:
def func_ivp(t, w):
    
    v = 0.001
    
    # update psi 
    psi = scipy.sparse.linalg.spsolve(A, w)
    
    dw = (v*A@w) - ((B@psi)*(C@w)) + ((C@psi)*(B@w))
    
    return dw 

In [18]:
tic_gaus = time.time()
sol = scipy.integrate.solve_ivp(lambda t, w: func_ivp(t, w), xp, w0, t_eval = t_evals)
toc_gaus = time.time()
print(toc_gaus-tic_gaus)

3.875181198120117


In [19]:
A7 = sol.y.T

### A8

In [20]:
L = 10
xp = [0, 4]

# time
t_evals = np.linspace(0,4,9)
dt = 0.5

In [21]:
w0_func = lambda x,y: np.exp((-2*(x**2))-((y**2)/20))
w0=np.zeros(len(x)**2)
k=0
for i in x:
    for j in y:
        w0[k]= w0_func(i,j)
        k=k+1

In [22]:
PLU = scipy.sparse.linalg.splu(A)

def func_ivp_plu(t, w):
    
    v = 0.001
    
    # update psi 
    psi = PLU.solve(w)
    
    dw = (v*A@w) - ((B@psi)*(C@w)) + ((C@psi)*(B@w))
    
    return dw 

/Users/annahuang/opt/anaconda3/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


In [23]:
tic_plu = time.time()
sol2 = scipy.integrate.solve_ivp(lambda t, w: func_ivp_plu(t, w), xp, w0, t_eval = t_evals)
toc_plu = time.time()
print(toc_plu-tic_plu)

0.10508370399475098


In [24]:
A8 = sol2.y.T

In [25]:
q = sol2.y[:,0].reshape(128,128)

In [26]:
w = sol2.y[:,1].reshape(128,128)

In [27]:
e = sol2.y[:,2].reshape(128,128)

In [28]:
r = sol2.y[:,3].reshape(128,128)

In [29]:
t = sol2.y[:,4].reshape(128,128)

In [30]:
y = sol2.y[:,5].reshape(128,128)

In [31]:
u = sol2.y[:,6].reshape(128,128)

In [32]:
i = sol2.y[:,7].reshape(128,128)

In [33]:
o = sol2.y[:,8].reshape(128,128)

In [34]:
A9 = np.stack([q,w,e,r,t,y,u,i,o])

In [35]:
# L = 10
# term = 4
# x = np.arange(-L, L, 0.3125)
# y = np.arange(-L, L, 0.3125)
# N = len(x)

In [36]:
# fig5, ax5 = plt.subplots()

# X, Y = np.meshgrid(x, y)
# surf = ax5.contourf(X, Y, w)
# fig5.colorbar(surf)

# # ax5.set_title('Contour plot of the time evolution of the second mode', fontsize = 16)
# ax5.set_xlabel(r'$x-value$', fontsize=14)
# ax5.set_ylabel(r'$y-value$', fontsize=14)